# Library Data Contest 2019
**Authors:** Haleh Ale-Ahmand, Moein Hosseini<br/>
March 2019

## 0. Packages and Functions

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os